In [247]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, clear_output

#import pip
#!pip install gspread
import gspread
import pandas as pd
import df2gspread as d2g
import random
import ipywidgets as widgets
from IPython.display import display, clear_output

from oauth2client.service_account import ServiceAccountCredentials


In [248]:
#load data 

scope = [
   'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

#Name of our Service Account Key
google_key_file = 'chooseadventureapp-07caf7abc2ef.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

spreadsheet_key = '18b4EVBQDiy4-cDF9Iq5lCtzcnz3AldyPyi8XmSgLvt0'
workbook = gc.open_by_key(spreadsheet_key)
sheet1=workbook.worksheet('Sheet1')
#!jupyter nbextension enable – py widgetsnbextension – sys-prefix
#!jupyter serverextension enable voila – sys-prefix

In [205]:
#pull in data from google doc

values = sheet1.get_all_values()
activities_list = pd.DataFrame(values [1:])
columns = values[0]
activities_list.columns = columns

In [250]:
#image widget
# Image Widget

file = open("hbd.jpg", "rb")
image = file.read()

image_headline = widgets.Image(
                    value=image,
                    format='jpg',
                    width='800'
                )

#label_headline = widgets.Label(
#                    value='Photo by CDC on Unsplash',
 #                   style={'description_width': 'initial'}
  #              )

vbox_headline = widgets.VBox([image_headline])#, label_headline])


In [206]:
#build widgets

weather = widgets.ToggleButtons(options = ['amazing','good','bad','not sure'])

time = widgets.ToggleButtons(options = ['AM','PM','AMPM'])

date = widgets.DatePicker(description='Pick a Date')

group = widgets.IntSlider(value=2,
                          min=1,
                          max=10,
                          step=1,
                          description="# of ppl"
                        )

cost = widgets.SelectionSlider(
    options=['0', '$', '$$', '$$$'],
    value='0',
    description='how bougie?',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
    )

goButton = widgets.Button(description = "Choose Adventure!",
                            tooltip='Go!',
                            button_style='primary'
                            )

out = widgets.Output(layout={'border': '1px solid black'})

In [208]:
#write function to choose activity based on user input 

def Choose(userInput):
    #need to end up with 1-3 results that match the user's criteria
    
    acts = activities_list
    weather = userInput[0]
    date = userInput[1]
    group = userInput[2]
    cost = userInput[3]
    time = userInput[4]
        
    #filter out any activity at the wrong time
    #am = get rid of pm, pm = get rid of am, always keep "both"    
    if time == 'PM':
            index_to_drop = acts[ (acts.AMPM == 'AM')].index 
            acts.drop(index_to_drop , inplace=True)
            
    elif time == 'AM':
            index_to_drop = acts[ (acts.AMPM == 'PM')].index
            acts.drop(index_to_drop , inplace=True)
        
    acts = acts[acts.AMPM == time] # really it just has to contain "am" or "pm"
    
    if weather == 'bad':
            acts = acts[acts.venue != 'outdoor']
    elif weather == 'amazing':
            acts = acts[acts.venue == 'outdoor']
    
    if len(cost) == 1:
        index_to_drop = acts[ (acts.cost == "$$" ) | (acts.cost == '$$$')].index
        acts.drop(index_to_drop, inplace=True)
    
    #filter by neighborhood
    
    return acts

In [235]:
def on_button_clicked(event):
    with out:
        answers = [weather.value, date.value, group.value, cost.value, time.value]
        clear_output()
        print("Here is your result...")
        print(f'the date is {date.value}.')
        print(f'the weather is {weather.value}.')
        print(f"you have {group.value} people.")        
        print(f'you have {cost.value} money.') 
        
        #loading bar?
        
        newList = Choose(answers)
        
        if len(newList) == 1:
            print(f'I found the perfect activity!')
            print(f'    do this: {newList.ActivityName.iloc[0]} at {newList.Address.iloc[0]}')
            print(f'    {newList.Website.iloc[0]}')
            
        elif len(newList) >1:
            rand_index = random.randint(0,len(newList)-1)
            print(f'I found {len(newList)} activities!')
            print(f'    do this: {newList.ActivityName.iloc[rand_index]} at {newList.LocationName.iloc[rand_index]} in {newList.neighborhood.iloc[rand_index]}') 
            print(f'    {newList.Address.iloc[rand_index]}')
            print(f'    {newList.Website.iloc[rand_index]}')
        
        else:
            print(f"I don't have the perfect activity for that... try different input!")
            print(f"Here's a random activity instead: " )
            rand_index = random.randint(0,len(activities_list)-1)
            print(f'    do this: {activities_list.ActivityName.iloc[rand_index]} at {activities_list.LocationName.iloc[rand_index]} in {activities_list.neighborhood.iloc[rand_index]}') 
            print(f'    {activities_list.Address.iloc[rand_index]}')
            print(f'    {activities_list.Website.iloc[rand_index]}')

    
goButton.on_click(on_button_clicked)
vbox_result = widgets.VBox([goButton, out])

In [252]:
# define text to display on the page
space = widgets.HTML(value='<br>')
text0 = widgets.HTML(value='<b><font size = 10> Hi beautiful! </b> So you want to do something fun? :)</font>')

text1 = widgets.HTML(value='How is the weather?')
text2 = widgets.HTML(value='Pick a date')
text3 = widgets.HTML(value='How big is your group?')
text4 = widgets.HTML(value='how much money you trying to spend?')
text5 = widgets.HTML(value='Is is day or night? (Choose AMPM if both are ok )')
text6 = widgets.HTML(value='Ready? Click button to choose your adventure!')


vbox_text = widgets.VBox([space, space, text0, space, text1, weather, text2, date, text3, group, text4, cost, text5, time, text6, vbox_result])


In [249]:
page = widgets.HBox([vbox_headline, vbox_text])
display(page)